# CRAWLING

> 비동기 방식 : 전체 페이지 중 일부만 바뀔 수 있도록 자바스크립트로 요청을 보낸다. 개발자 모드의 네트워크 탭에서 요청 내용 확인 가능

> 개발자 도구 탭 
- Elements - HTML 구조
- Console - Javascript 디버깅
- Sources - 웹페이지를 구성하는 src
- Performance - 웹페이지 성능 체크
- Network - 웹페이지에서 요청한 파일
- Memory - 웹페이지 메모리 사용률
- Application - 브라우저 스토리지정보 (Storage, Session, Cookie)

> STATUS CODE
- 200 정상 작동
- 301 리다이렉트
- 404 사용자 오류
- 502 서버 오류

cf. urlencoding 

## 동적 웹 크롤링

In [1]:
import requests

In [16]:
headers = {'User-Agent' :"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

In [17]:
response = requests.get("https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=101&sid2=259", headers=headers)

In [8]:
response = requests.get("https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=101&sid2=259", 
                        headers=headers 
                        # params={
                        #     'mode':'LS2D'
                        #     'mid' : 'shm',
                        #     'sid1' : 101,
                        #     'sid2' :259
                        # }
                        )
# requests.post()
# requests.put()
# requests.delete()  요청에 따라 다양한 메서드 사용 가능

AttributeError: 'set' object has no attribute 'items'

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
headers 정보가 없을 때 봇으로 인식하고 뜨는 에러

In [20]:
print(response.status_code)


200


In [19]:
print(response.text)





<!DOCTYPE HTML>
<html lang="ko">
<head>
<meta charset="euc-kr">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="referrer" contents="always">
<meta http-equiv="refresh" content="600" />
<meta name="viewport" content="width=1106" />

    
    
        
            
                
                    
                        
                        
                        
                    
                    
                    
                    
                    
                
            
            
            
            
            
        
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

<meta property="og:title"       content="경제 금융 : 네이버뉴스">
<meta property="og:type"        content="website">
<meta property="og:url"         content="https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=101&sid2=259">
<meta property="og:image"       content="https://ssl.pstatic.net/static.news/image/news/ogtag

In [13]:
from bs4 import BeautifulSoup

In [21]:
soup = BeautifulSoup(response.text, 'html.parser')

In [22]:
soup


<!DOCTYPE HTML>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta contents="always" name="referrer"/>
<meta content="600" http-equiv="refresh">
<meta content="width=1106" name="viewport">
<meta content="경제 금융 : 네이버뉴스" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://news.naver.com/main/list.naver?mode=LS2D&amp;mid=shm&amp;sid1=101&amp;sid2=259" property="og:url"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_800x420_20221201.png" property="og:image">
<meta content="각 언론사별 금융 기사 제공" property="og:description"/>
<meta content="네이버" property="og:article:author">
<meta content="summary" name="twitter:card"/>
<meta content="경제 금융 : 네이버뉴스" name="twitter:title"/>
<meta content="네이버 뉴스" name="twitter:site"/>
<meta content="네이버 뉴스" name="twitter:creator"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_800x420_20221201.png" name="tw

div 스페이스바로 나뉘어져 있으면 각각 다른 클래스

In [42]:
# newsDivTag = soup.select_one('div.list_body')
newsDivTag = soup.select_one('div.list_body.newsflash_body') # list_body newsflash_body -> list_body.newsflash_body 로 변경 필수

In [43]:
newsDivTag

<div class="list_body newsflash_body">
<ul class="type06_headline">
<li>
<dl>
<dt class="photo">
<a href="https://n.news.naver.com/mnews/article/421/0006924991?sid=101">
<img alt="리치앤코, ‘굿리치’로 사명 변경…대고객 인지도 높은 사명 채택" height="72" onerror="javascript:this.src='https://ssl.pstatic.net/static.news/image/news/2009/noimage_106x72.png';" src="https://imgnews.pstatic.net/image/origin/421/2023/07/13/6924991.jpg?type=nf106_72" width="106"/>
</a>
</dt>
<dt>
<a href="https://n.news.naver.com/mnews/article/421/0006924991?sid=101">
								 리치앤코, ‘굿리치’로 사명 변경…대고객 인지도 높은 사명 채택
								</a>
</dt>
<dd>
<span class="lede">리치앤코가 굿리치 주식회사로 간판을 바꿔달고 본격적인 새 출발에 나선다고 13일 밝혔다. 리치앤코는 지난달 23일 임시 주주총회 …</span>
<span class="writing">뉴스1</span>
<span class="date is_new">1분전</span>
</dd>
</dl>
</li>
<li>
<dl>
<dt class="photo">
<a href="https://n.news.naver.com/mnews/article/417/0000934746?sid=101">
<img alt="[머니S포토] '금융시장 안정 방점' 금통위, 기준금리 3.50% 동결" height="72" onerror="javascript:this.src='https://ssl.pstatic.net/

In [44]:
liTags = newsDivTag.select('li')
liTags

[<li>
 <dl>
 <dt class="photo">
 <a href="https://n.news.naver.com/mnews/article/421/0006924991?sid=101">
 <img alt="리치앤코, ‘굿리치’로 사명 변경…대고객 인지도 높은 사명 채택" height="72" onerror="javascript:this.src='https://ssl.pstatic.net/static.news/image/news/2009/noimage_106x72.png';" src="https://imgnews.pstatic.net/image/origin/421/2023/07/13/6924991.jpg?type=nf106_72" width="106"/>
 </a>
 </dt>
 <dt>
 <a href="https://n.news.naver.com/mnews/article/421/0006924991?sid=101">
 								 리치앤코, ‘굿리치’로 사명 변경…대고객 인지도 높은 사명 채택
 								</a>
 </dt>
 <dd>
 <span class="lede">리치앤코가 굿리치 주식회사로 간판을 바꿔달고 본격적인 새 출발에 나선다고 13일 밝혔다. 리치앤코는 지난달 23일 임시 주주총회 …</span>
 <span class="writing">뉴스1</span>
 <span class="date is_new">1분전</span>
 </dd>
 </dl>
 </li>,
 <li>
 <dl>
 <dt class="photo">
 <a href="https://n.news.naver.com/mnews/article/417/0000934746?sid=101">
 <img alt="[머니S포토] '금융시장 안정 방점' 금통위, 기준금리 3.50% 동결" height="72" onerror="javascript:this.src='https://ssl.pstatic.net/static.news/image/news/2009/noimage_106x72.

In [45]:
len(liTags)

20

In [46]:
liTags[0].select('dt')

[<dt class="photo">
 <a href="https://n.news.naver.com/mnews/article/421/0006924991?sid=101">
 <img alt="리치앤코, ‘굿리치’로 사명 변경…대고객 인지도 높은 사명 채택" height="72" onerror="javascript:this.src='https://ssl.pstatic.net/static.news/image/news/2009/noimage_106x72.png';" src="https://imgnews.pstatic.net/image/origin/421/2023/07/13/6924991.jpg?type=nf106_72" width="106"/>
 </a>
 </dt>,
 <dt>
 <a href="https://n.news.naver.com/mnews/article/421/0006924991?sid=101">
 								 리치앤코, ‘굿리치’로 사명 변경…대고객 인지도 높은 사명 채택
 								</a>
 </dt>]

In [47]:
liTags[0].select('dt')[1]

<dt>
<a href="https://n.news.naver.com/mnews/article/421/0006924991?sid=101">
								 리치앤코, ‘굿리치’로 사명 변경…대고객 인지도 높은 사명 채택
								</a>
</dt>

In [50]:
liTags[0].select('dt')[1].text # 기사 제목 가져오기

'\n\n\t\t\t\t\t\t\t\t 리치앤코, ‘굿리치’로 사명 변경…대고객 인지도 높은 사명 채택\n\t\t\t\t\t\t\t\t\n'

In [51]:
liTags[0].select('dt')[1].text.strip()

'리치앤코, ‘굿리치’로 사명 변경…대고객 인지도 높은 사명 채택'

In [55]:
titles = []
for liTag in liTags:
    title = liTag.select('dt')[-1].text.strip() # ('dt')[1]로 설정할 경우 사진이 없는 기사도 있기 때문에 IndexError: list index out of range 에러남 -> ('dt')[-1] 로 변경
    titles.append(title)
titles

['리치앤코, ‘굿리치’로 사명 변경…대고객 인지도 높은 사명 채택',
 "[머니S포토] '금융시장 안정 방점' 금통위, 기준금리 3.50% 동결",
 '에이플러스그룹, 진단기기업체 나노엔텍 지분 인수',
 '[속보] 한은, 기준금리 4연속 동결…한미금리차 2%p 임박',
 '[머니S포토] 하반기 첫 금통위 입장하는 이창용 한은 총재',
 "[머니S포토] 금통위, 기준금리 3.50%…금융불안에 '4연속' 동결",
 '[속보] 한은, 기준금리 연 3.50% 유지… 경기불안 고려 4회 연속 동결',
 '“제2 카카오 화재 막아야” 복구센터 의무 늘리고 책임보험 최저한도도 상향',
 '美 디스인플레이션 모드…"7월 금리인상 끝이 보인다"',
 '금통위, 금융 불안에 기준금리 4연속 동결(상보)',
 'KB손보, 경영전략회의 개최...김기환 사장 "일류기업 되자" 강조',
 '금감원 "금융권 IT 내부통제 여전히 부족…가이드라인 제정"',
 '[3보] 기준금리 3.5%로 4회 연속 동결…이제 `물가`보다 `성장`',
 '[동정] 과기정통부-KB국민은행, AI 윤리기준 확산 논의',
 '[속보] “금융시장 안정 방점”...한은, 기준금리 4회 연속 동결…연 3.50%',
 '삼성운용, \'달러 MMF\' 국내 첫 출시…"연 5% 이상 수익"',
 '"카카오 같은 화재 발생해도 금융사 업무연속성 확보해야"',
 '한은, 기준금리 3.50%로 4연속 동결..한미금리차 1.75%p',
 '한은 금통위, 기준금리 3.5% 동결…경기·금융 안정에 무게',
 "한은 4연속 동결에…이달 말 '2%p 한미 금리차' 개봉박두"]